# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/ryan.so/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ryan.so/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/ryan.so/personal/AIE8-RyanSo/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/ryan.so/personal/AIE8-RyanSo/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/ryan.so/personal/AIE8-RyanSo/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '693c21'. Skipping!
Property 'summary' already exists in node 'a29be8'. Skipping!
Property 'summary' already exists in node '92a23d'. Skipping!
Property 'summary' already exists in node '56277c'. Skipping!
Property 'summary' already exists in node 'd15bc9'. Skipping!
Property 'summary' already exists in node '3ad30d'. Skipping!
Property 'summary' already exists in node 'e0391f'. Skipping!
Property 'summary' already exists in node '6306b8'. Skipping!
Property 'summary' already exists in node '8edc96'. Skipping!
Property 'summary' already exists in node 'ad4187'. Skipping!
Property 'summary' already exists in node 'f670f2'. Skipping!
Property 'summary' already exists in node 'dd0b24'. Skipping!
Property 'summary' already exists in node '2d775d'. Skipping!
Property 'summary' already exists in node 'c823cb'. Skipping!
Property 'summary' already exists in node 'a46ebe'. Skipping!
Property 'summary' already exists in node '65d594'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '65d594'. Skipping!
Property 'summary_embedding' already exists in node '693c21'. Skipping!
Property 'summary_embedding' already exists in node 'c94c62'. Skipping!
Property 'summary_embedding' already exists in node '3ad30d'. Skipping!
Property 'summary_embedding' already exists in node 'e0391f'. Skipping!
Property 'summary_embedding' already exists in node 'a29be8'. Skipping!
Property 'summary_embedding' already exists in node '92a23d'. Skipping!
Property 'summary_embedding' already exists in node '56277c'. Skipping!
Property 'summary_embedding' already exists in node 'd15bc9'. Skipping!
Property 'summary_embedding' already exists in node '6306b8'. Skipping!
Property 'summary_embedding' already exists in node 'ad4187'. Skipping!
Property 'summary_embedding' already exists in node '8edc96'. Skipping!
Property 'summary_embedding' already exists in node 'dd0b24'. Skipping!
Property 'summary_embedding' already exists in node '2d775d'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 711)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 711)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


The three types of query synthesizers are:
1. SingleHopSpecificQuerySynthesizer:
* Creates straightforward, direct questions that can be answered from a single piece of information or document
* These are simple queries that require only one "hop" through the knowledge graph
* Example: "What is ChatGPT?" or "How many users does OpenAI have?"
2. MultiHopAbstractQuerySynthesizer:
* Generates complex, abstract questions that require synthesizing information from multiple sources
* These questions need multiple "hops" through the knowledge graph to connect different concepts
* The answers tend to be more conceptual and require reasoning across different pieces of information
* Example: "How does user behavior relate to economic impact across different industries?"
3. MultiHopSpecificQuerySynthesizer:
* Creates complex questions that require specific information from multiple sources
* Like MultiHopAbstract, these need multiple "hops" but focus on concrete, specific details rather than abstract concepts
* Example: "Based on the data from June 2024, how does ChatGPT usage in the US compare to global trends?"

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What are Imas in the context of ChatGPT usage?,[Introduction ChatGPT launched in November 202...,The provided context does not specify what Ima...,single_hop_specifc_query_synthesizer
1,How does the concept of Collis relate to the a...,[Table 1: ChatGPT daily message counts (millio...,The provided context discusses ChatGPT's daily...,single_hop_specifc_query_synthesizer
2,How does the usage of ChatGPT vary among users...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 shows that u...,single_hop_specifc_query_synthesizer
3,Relationships how they help?,[Conclusion This paper studies the rapid growt...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
4,how user behavior and changing usage patterns ...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,the context indicates that from june 2024 to j...,multi_hop_abstract_query_synthesizer
5,Based on the data from June 2024 and June 2025...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data shows that from June 2024 to June 202...,multi_hop_abstract_query_synthesizer
6,Based on the data showing that non-work messag...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data indicates that from June 2024 to June...,multi_hop_abstract_query_synthesizer
7,"How does the popularity of Claude, as indicate...",[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The data shows that ChatGPT's overall message ...,multi_hop_specific_query_synthesizer
8,How does the Claude platform compare to ChatGP...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context indicates that ChatGPT has experie...,multi_hop_specific_query_synthesizer
9,"US ChatGPT use mainly for work or non-work, an...",[<1-hop>\n\nConclusion This paper studies the ...,"The context shows that in the US, about 70% of...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/37 [00:00<?, ?it/s]

Property 'summary' already exists in node '2252cd'. Skipping!
Property 'summary' already exists in node '087b04'. Skipping!
Property 'summary' already exists in node 'fb00a8'. Skipping!
Property 'summary' already exists in node 'e82601'. Skipping!
Property 'summary' already exists in node '3914ff'. Skipping!
Property 'summary' already exists in node 'beaeb1'. Skipping!
Property 'summary' already exists in node 'e4163d'. Skipping!
Property 'summary' already exists in node '80cfc3'. Skipping!
Property 'summary' already exists in node '7f29fd'. Skipping!
Property 'summary' already exists in node '3dab7f'. Skipping!
Property 'summary' already exists in node 'ed1f23'. Skipping!
Property 'summary' already exists in node 'b3a953'. Skipping!
Property 'summary' already exists in node 'f4c185'. Skipping!
Property 'summary' already exists in node 'cc9f89'. Skipping!
Property 'summary' already exists in node 'c2b046'. Skipping!
Property 'summary' already exists in node 'e362e6'. Skipping!


...

Property 'summary_embedding' already exists in node 'e82601'. Skipping!
Property 'summary_embedding' already exists in node '2252cd'. Skipping!
Property 'summary_embedding' already exists in node 'beaeb1'. Skipping!
Property 'summary_embedding' already exists in node '087b04'. Skipping!
Property 'summary_embedding' already exists in node 'fb00a8'. Skipping!
Property 'summary_embedding' already exists in node '3914ff'. Skipping!
Property 'summary_embedding' already exists in node 'e4163d'. Skipping!
Property 'summary_embedding' already exists in node '3dab7f'. Skipping!
Property 'summary_embedding' already exists in node '7f29fd'. Skipping!
Property 'summary_embedding' already exists in node 'f4c185'. Skipping!
Property 'summary_embedding' already exists in node '80cfc3'. Skipping!
Property 'summary_embedding' already exists in node 'ed1f23'. Skipping!
Property 'summary_embedding' already exists in node 'e362e6'. Skipping!
Property 'summary_embedding' already exists in node 'b3a953'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Who is Acemoglu?,[Introduction ChatGPT launched in November 202...,The provided context does not include any info...,single_hop_specifc_query_synthesizer
1,What role does OpenAI play in AI development?,[Table 1: ChatGPT daily message counts (millio...,The context does not specify the exact role of...,single_hop_specifc_query_synthesizer
2,How does ChatGPT provide economic value?,"[Doing, and that Asking messages are consisten...",ChatGPT likely improves worker output by provi...,single_hop_specifc_query_synthesizer
3,What does Ide refer to in the context of AI ag...,"[How does ChatGPT provide economic value, and ...","In the context of AI agents like ChatGPT, Ide ...",single_hop_specifc_query_synthesizer
4,How does ChatGPT's role in providing decision ...,"[<1-hop>\n\nDoing, and that Asking messages ar...",ChatGPT likely improves worker output by offer...,multi_hop_abstract_query_synthesizer
5,How do differences in message types (asking vs...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context indicates that users in highly pai...,multi_hop_abstract_query_synthesizer
6,"ChatGPT use for writing and text modif ask, ho...",[<1-hop>\n\nConclusion This paper studies the ...,The context explains that about 42% of message...,multi_hop_abstract_query_synthesizer
7,Hw does ChatGPT provide econmic valyu and for ...,"[<1-hop>\n\nDoing, and that Asking messages ar...",ChatGPT provides economic value by improving w...,multi_hop_abstract_query_synthesizer
8,"Based on the data, how does ChatGPT's usage in...",[<1-hop>\n\nConclusion This paper studies the ...,The context indicates that ChatGPT's usage in ...,multi_hop_specific_query_synthesizer
9,wHAT US is the most common use of ChatGPT?,[<1-hop>\n\nConclusion This paper studies the ...,The most common use of ChatGPT in the US is Pr...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [17]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [18]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [19]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [22]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [23]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [24]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [26]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [27]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in various ways both at work and outside of work. AI is employed to perform workplace tasks, either by augmenting or automating human labor. Users seek AI for producing writing, software code, spreadsheets, and other digital products, which distinguishes generative AI from traditional web search engines. Additionally, AI is used for obtaining information and advice, self-expression such as relationships and personal reflection, and even games and role play. The usage spans from acting as co-workers producing output to co-pilots that provide advice and improve human productivity in problem-solving. Overall, AI usage is flexible, involving asking questions, doing tasks, and expressing ideas.\n\nIn summary, people are using AI to:\n\n- Augment or automate workplace tasks\n- Produce digital content like writing and code\n- Obtain information and advice\n- Engage in self-expression and personal ref

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [28]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [29]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
    * Evaluates the correctness of question-answering responses
    * Compares the generated answer against the expected/reference answer
    * Measures how accurately the RAG system answers questions based on the provided context
- `labeled_helpfulness_evaluator`:
    * Evaluates the helpfulness of responses to users
    * Takes into account the correct reference answer when determining if the submission is helpful
    * Assesses whether the response would actually be useful to someone asking the question
    * Uses labeled criteria evaluation with the specific criterion of helpfulness
- `dopeness_evaluator`:
    * Evaluates the creativity and engagement of responses
    * Determines if responses are "dope, lit, cool" or just generic/boring
    * Measures whether the response has personality and avoids generic, templated answers
    * This is a custom evaluator designed to test if the improved prompt makes responses more engaging

## LangSmith Evaluation

In [30]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'kind-square-48' at:
https://smith.langchain.com/o/efb53ee0-b891-4203-a7bd-4250842b1fc4/datasets/736a96db-f78c-4471-a826-9c06aa94d201/compare?selectedSessions=e6344e68-bbdc-4e11-88f3-44b3ead08ec0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Based on the evaluation of ChatGPT's economic ...,"Based on the provided context, OpenAI's develo...",None,OpenAI's development of large language models ...,1,1,0,3.706642,3482cc84-7cf2-4be5-bca6-cf16de1ce905,f1e4db0b-3089-4252-8f15-881921975920
1,Based on how ChatGPT provides economic value b...,"Based on the context, ChatGPT provides economi...",None,"OpenAI's development of multiple models, such ...",1,1,0,3.481403,8d940e28-db8a-4b6e-994d-cf9ccfbc94f3,d8bb8d21-ab4b-4009-9273-1a1b993a94fe
2,wHAT US is the most common use of ChatGPT?,The most common use of ChatGPT is the exchange...,None,The most common use of ChatGPT in the US is Pr...,0,0,0,1.419657,50f4ee9f-ca7f-4dbd-b06e-6260704db3d3,2803fe98-4d60-4054-9d2d-4ffb288aa548
3,"Based on the data, how does ChatGPT's usage in...","Based on the provided context, ChatGPT has exp...",None,The context indicates that ChatGPT's usage in ...,1,1,0,4.724967,757edc52-275f-4815-8564-1bbaf9124bbf,81cc4afd-f494-4675-8b6c-10a786e5117b
4,Hw does ChatGPT provide econmic valyu and for ...,"Based on the context provided, ChatGPT provide...",None,ChatGPT provides economic value by improving w...,1,0,0,1.686216,24434fb4-6616-4ee4-bed1-bbf1c9b9e1d6,cca6ba1a-6ad4-4500-afdc-c416bca59226
5,"ChatGPT use for writing and text modif ask, ho...","Based on the provided context, requests relate...",None,The context explains that about 42% of message...,1,1,0,2.727790,7649f7af-ab66-4c2d-a033-2b78a5ff49b6,f31692d1-8189-4460-9b73-b07efdba1bc9
6,How do differences in message types (asking vs...,I don't know.,None,The context indicates that users in highly pai...,0,0,0,0.901353,51adaebe-7c5c-4d6d-9983-4de70e643add,b6e26790-0c99-4067-9e74-fcaa8046f4b9
7,How does ChatGPT's role in providing decision ...,"Based on the provided context, ChatGPT enhance...",None,ChatGPT likely improves worker output by offer...,1,1,0,3.674421,5db6cfe7-b911-4d55-a555-d91cd8282031,7741e583-5af9-4db1-b484-8c1f685d5c5a
8,What does Ide refer to in the context of AI ag...,Ide and Talamas (2025) develop a model where A...,None,"In the context of AI agents like ChatGPT, Ide ...",1,1,0,1.965878,6ca6cad4-312e-4101-bfa0-d748457c86c3,54223740-6048-44d1-a3dd-3bfd614e3143
9,How does ChatGPT provide economic value?,ChatGPT provides economic value primarily thro...,None,ChatGPT likely improves worker output by provi...,1,1,0,1.811204,b50c37d8-8f96-4733-83c3-e56faf1b4278,9464dbfc-3053-4585-96b4-5229558aae6b


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [31]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [32]:
rag_documents = docs

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

Modifying chunk size from 500 to 1000 characters can significantly impact RAG performance in several ways:

Positive Impacts of Larger Chunks (1000 vs 500):
* More Context per Chunk: Larger chunks contain more complete information, reducing the risk of important context being split across multiple chunks. This helps the LLM better understand the full context when generating answers.
* Better Semantic Coherence: Larger chunks are more likely to contain complete thoughts, sentences, and concepts, making them more semantically meaningful units for retrieval.
* Reduced Fragmentation: With larger chunks, related information is less likely to be separated, which means when a relevant chunk is retrieved, it's more likely to contain all the necessary information to answer a question.
* Fewer Total Chunks: Larger chunks mean fewer total chunks in the vector database, which can improve retrieval efficiency and reduce the chance of retrieving partially relevant information.

Potential Drawbacks:
* Less Granular Retrieval: Larger chunks might include some irrelevant information alongside the relevant parts, potentially introducing noise.
* Increased Token Usage: Larger chunks mean more tokens sent to the LLM, increasing costs and potentially hitting context limits.
* Potentially Less Precise Matching: Smaller chunks might allow for more precise semantic matching to specific questions.

In this case, the increase from 500 to 1000 characters likely improved performance because the documents contain complex, interconnected information about ChatGPT usage patterns that benefits from having more complete context in each chunk.

In [34]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

Changing the embedding model from text-embedding-3-small to text-embedding-3-large can improve RAG performance for several key reasons:

Technical Improvements of text-embedding-3-large:
* Higher Dimensional Representations: The "large" model typically produces higher-dimensional embeddings (e.g., 3072 dimensions vs 1536 for small), allowing for more nuanced and detailed semantic representations of text.
* Better Semantic Understanding: Larger embedding models are trained with more parameters and data, enabling them to capture more subtle semantic relationships, context, and meaning in text.
* Improved Similarity Matching: With more sophisticated embeddings, the model can better distinguish between genuinely similar content and superficially similar but semantically different content.
* Better Handling of Complex Queries: The larger model can better understand complex, multi-faceted queries and match them with the most relevant chunks, especially important for the multi-hop queries generated by RAGAS.

Impact on RAG Performance:
* More Accurate Retrieval: Better embeddings lead to more accurate similarity calculations, meaning the retriever is more likely to find the chunks that actually contain the information needed to answer the question.
* Reduced Noise: Better semantic understanding reduces the likelihood of retrieving chunks that are topically related but don't actually help answer the specific question.
* Better Cross-Domain Understanding: For documents covering multiple topics (like the ChatGPT usage data), the large model better understands how different concepts relate to each other.

Trade-offs:
* Cost: Larger embedding models are more expensive to run
* Latency: May be slightly slower due to increased computational requirements
* Storage: Higher-dimensional embeddings require more storage space

In this specific use case, the upgrade likely improved performance because the documents contain complex relationships between user behavior, economic impact, and usage patterns that benefit from the more sophisticated semantic understanding of the larger model.

In [35]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [36]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [37]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [38]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, let’s crank this up to eleven on the rad meter! Based on the slick context provided, people aren’t just using AI as some boring, soulless task-doer—they’re tapping into ChatGPT and generative AI as *decision-making sidekicks* and *research wingmen* that supercharge their brainpower at work. \n\nHere’s the deal: AI isn’t just punching in the hours doing grunt tasks; it’s *amplifying human smarts* especially in knowledge-heavy gigs where every call and move demands razor-sharp decisions. Think of it like having a turbocharged advisor in your corner, helping you dodge pitfalls and make boss-level moves that boost productivity and ultimately fatten those paychecks. \n\nPlus, the sheer economic muscle is wild—US users would pay almost $100 just to keep using generative AI every month, reflecting a massive multi-billion-dollar value from this tech helping folks cash in smarter and faster. So when it comes to making money, AI’s not just clocking in; it’s making you a brainier, decis

Finally, we can evaluate the new chain on the same test set!

In [39]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'extraneous-error-8' at:
https://smith.langchain.com/o/efb53ee0-b891-4203-a7bd-4250842b1fc4/datasets/736a96db-f78c-4471-a826-9c06aa94d201/compare?selectedSessions=e3abe7e2-ada7-4351-9a38-aae55730df53




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Based on the evaluation of ChatGPT's economic ...,"Alright, let’s crank this up to eleven and bre...",None,OpenAI's development of large language models ...,1,1,1,10.397068,3482cc84-7cf2-4be5-bca6-cf16de1ce905,f0935f34-57a8-4886-9ce1-80c12ab23301
1,Based on how ChatGPT provides economic value b...,"Yo, let’s break down this economic flex that O...",None,"OpenAI's development of multiple models, such ...",1,1,1,5.360453,8d940e28-db8a-4b6e-994d-cf9ccfbc94f3,0af80dac-e644-4687-ba2c-5a3ef7509d00
2,wHAT US is the most common use of ChatGPT?,"Alright, buckle up — the dopest insight straig...",None,The most common use of ChatGPT in the US is Pr...,0,0,1,3.988958,50f4ee9f-ca7f-4dbd-b06e-6260704db3d3,2f94ae57-8b1f-41df-a362-85a0c1284eab
3,"Based on the data, how does ChatGPT's usage in...","Alright, buckle up for this dopest take on Cha...",None,The context indicates that ChatGPT's usage in ...,1,1,1,5.321696,757edc52-275f-4815-8564-1bbaf9124bbf,951696d8-0627-4b4e-999d-e1467c919230
4,Hw does ChatGPT provide econmic valyu and for ...,"Alright, let’s drop some knowledge bombs strai...",None,ChatGPT provides economic value by improving w...,1,1,1,3.665798,24434fb4-6616-4ee4-bed1-bbf1c9b9e1d6,a9ce6791-a99c-42b6-8896-a29d07686b2c
5,"ChatGPT use for writing and text modif ask, ho...","Yo, here’s the deal straight from the AI stree...",None,The context explains that about 42% of message...,1,1,1,4.684967,7649f7af-ab66-4c2d-a033-2b78a5ff49b6,ccbac15d-6ea8-40a5-8c48-e3f72b77f15a
6,How do differences in message types (asking vs...,"Yo, diving into the context, the docs break do...",None,The context indicates that users in highly pai...,0,0,1,5.355372,51adaebe-7c5c-4d6d-9983-4de70e643add,23cfb20b-8c5b-4a73-b859-3f891e2ad7e5
7,How does ChatGPT's role in providing decision ...,"Alright, let’s break this down with some turbo...",None,ChatGPT likely improves worker output by offer...,1,1,1,7.031114,5db6cfe7-b911-4d55-a555-d91cd8282031,20a24267-653f-4ae6-9654-01a87ff47b83
8,What does Ide refer to in the context of AI ag...,"Oh, heck yes—let's drop some serious knowledge...",None,"In the context of AI agents like ChatGPT, Ide ...",1,1,1,4.902001,6ca6cad4-312e-4101-bfa0-d748457c86c3,8e3e8d1c-33ad-4e7c-9689-dfa9fa70303e
9,How does ChatGPT provide economic value?,"Yo, here’s the lowdown straight from the digit...",None,ChatGPT likely improves worker output by provi...,1,1,1,2.660456,b50c37d8-8f96-4733-83c3-e56faf1b4278,4c232995-e3c2-479b-b172-48c4b995799b


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![Chain 1 Results](chain_1.png)

![Chain 2 Results](chain_2.png)

Performance Comparison Analysis

First Chain (Basic RAG) Results:
* Correctness: 7/12 questions scored 1 (58% accuracy)
* Helpfulness: 7/12 questions scored 1 (58% helpful)
* Dopeness: 0/12 questions scored 1 (0% dope)

Second Chain (Improved "Dopeness" RAG) Results:
* Correctness: 8/12 questions scored 1 (67% accuracy)
* Helpfulness: 8/12 questions scored 1 (67% helpful)
* Dopeness: 11/12 questions scored 1 (92% dope)

Analysis of Metric Changes:
1. Correctness Improved (58% → 67%)
    Why it improved:
    * Larger chunks (500→1000 chars): Provided more complete context, reducing information fragmentation
    * Better embedding model (small→large): More accurate retrieval of relevant information
    * Combined effect: Better retrieval + more context = more accurate answers
2. Helpfulness Improved (58% → 67%)
    Why it improved:
    * More comprehensive answers: Larger chunks allowed for more complete responses
    * Better context matching: Superior embeddings led to more relevant information retrieval
    * Maintained accuracy: The slight improvement in correctness contributed to helpfulness
3. Dopeness Dramatically Improved (0% → 92%)

    Why it dramatically improved:
    * Explicit prompt modification: The new prompt specifically instructed the model to "Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses"
    * Direct targeting: This was the primary goal of the prompt change, and it worked as intended
    * Style vs. content: This metric measures presentation style rather than factual accuracy, so the prompt change had maximum impact
4. Why some questions still scored 0 on correctness/helpfulness:
    * Some questions may have been about information not well-covered in the source documents
    * Even with improvements, some complex multi-hop questions remained challenging
    * The synthetic nature of the questions may have created some that are inherently difficult to answer from the available context
    
Key Insights:
* Technical improvements (chunk size, embeddings) provided modest but meaningful gains in accuracy and helpfulness
* Prompt engineering had the most dramatic impact on the targeted metric (dopeness)
* The improvements were complementary: better retrieval enabled the model to give both more accurate AND more engaging responses
* Trade-off consideration: The "dopeness" improvement came with increased response length and potentially higher token costs, as seen in the more verbose, colorful responses